In [ ]:
# ============================================================
# Colab script: run the repeated 10-fold CV experiment,
# record individual-level predictions + squared errors.
#
# Output CSV (individual-level):
#   pat_id, rep, fold, truth,
#   lasso_pred, llm_lasso_pred, spike_slab_mcmc_fixed_mpm_pred, llm_spike_slab_mcmc_fixed_mpm_pred,
#   lasso_sq_error, llm_lasso_sq_error, spike_slab_mcmc_fixed_mpm_sq_error, llm_spike_slab_mcmc_fixed_mpm_sq_error
#
# ============================================================

# -------------------------
# 0) Mount Drive + locate files
# -------------------------
from google.colab import drive
drive.mount("/content/drive")

import os, glob
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score

def find_file_in_drive(filename, base="/content/drive/MyDrive"):
    """
    Tries:
      1) base/filename
      2) recursive search base/**/filename
    Returns full path or raises.
    """
    p1 = os.path.join(base, filename)
    if os.path.exists(p1):
        return p1
    hits = glob.glob(os.path.join(base, "**", filename), recursive=True)
    if len(hits) > 0:
        # pick the shortest path (often the "closest" hit)
        hits = sorted(hits, key=len)
        return hits[0]
    raise FileNotFoundError(f"Could not find {filename} under {base}")

# EDIT only if the files are named differently
DATA_FILENAME    = "t60_reg_data.csv"
WEIGHTS_FILENAME = "aki_weights_60.csv"

DATA_PATH    = find_file_in_drive(DATA_FILENAME)
WEIGHTS_PATH = find_file_in_drive(WEIGHTS_FILENAME)

# Where to save outputs
OUT_DIR = "/content/drive/MyDrive"
os.makedirs(OUT_DIR, exist_ok=True)

ID_COL = "pat_id"
TARGET = "creatinine_t60"

print("DATA_PATH   =", DATA_PATH)
print("WEIGHTS_PATH=", WEIGHTS_PATH)
print("OUT_DIR     =", OUT_DIR)

# -------------------------
# 1) Load + subset
# -------------------------
df = pd.read_csv(DATA_PATH)
wdf = pd.read_csv(WEIGHTS_PATH)

need_cols = ["pvd", "current_smoker", "former_smoker"]
missing = [c for c in need_cols + [ID_COL, TARGET] if c not in df.columns]
assert len(missing) == 0, f"Missing required columns in data: {missing}"

df = df[(df["pvd"] == 1) & ((df["current_smoker"] == 1) | (df["former_smoker"] == 1))].copy()
df = df.drop(columns=["pvd", "current_smoker", "former_smoker"], errors="ignore")
print("[subset] n =", len(df), "| p_total (incl target/id) =", df.shape[1])

# -------------------------
# 2) Normalize weight columns to: value, importance
# -------------------------
if "value" not in wdf.columns:
    for cand in ["variable_name", "feature", "name"]:
        if cand in wdf.columns:
            wdf = wdf.rename(columns={cand: "value"})
            break
if "importance" not in wdf.columns:
    for cand in ["importance_score", "score", "weight"]:
        if cand in wdf.columns:
            wdf = wdf.rename(columns={cand: "importance"})
            break
assert "value" in wdf.columns and "importance" in wdf.columns, f"weights columns are {wdf.columns.tolist()}"

FEATURES = [c for c in df.columns if c not in [ID_COL, TARGET]]
P_FULL = len(FEATURES)
print("[design] n =", len(df), "p =", P_FULL)

# -------------------------
# 3) LLM importance calibration
# -------------------------
def calibrate_importance_to_01_to_1(raw_imp: np.ndarray, min_val=0.1) -> np.ndarray:
    x = np.asarray(raw_imp, float)
    lo, hi = np.nanmin(x), np.nanmax(x)
    if not np.isfinite(lo) or not np.isfinite(hi) or (hi - lo) < 1e-12:
        return np.ones_like(x)
    z = (x - lo) / (hi - lo)
    return min_val + (1.0 - min_val) * z

def penalty_factors_from_importance(imp_01_1: np.ndarray, power: float, eps=1e-6) -> np.ndarray:
    imp = np.clip(np.asarray(imp_01_1, float), eps, None)
    pen = (1.0 / imp) ** power
    return pen / np.mean(pen)

wmap = dict(zip(wdf["value"].astype(str), wdf["importance"].astype(float)))
raw_imp_all = np.array([wmap.get(f, np.nan) for f in FEATURES], dtype=float)
match_rate = np.isfinite(raw_imp_all).mean()
print(f"[LLM weight alignment] match_rate={match_rate:.3f}  missing={(~np.isfinite(raw_imp_all)).sum()}/{P_FULL}")

fallback = np.nanmedian(raw_imp_all) if np.isfinite(np.nanmedian(raw_imp_all)) else 1.0
raw_imp_all = np.where(np.isfinite(raw_imp_all), raw_imp_all, fallback)
imp_all_01_1 = calibrate_importance_to_01_to_1(raw_imp_all, min_val=0.1)
imp_map_full = {f: imp_all_01_1[i] for i, f in enumerate(FEATURES)}

# -------------------------
# 4) Fold preprocessing (train-only stats)
# -------------------------
def preprocess_fold(df_train: pd.DataFrame, df_val: pd.DataFrame, feature_names):
    Xtr_df = df_train[feature_names].copy()
    Xva_df = df_val[feature_names].copy()

    ytr = df_train[TARGET].values.reshape(-1, 1)
    yva = df_val[TARGET].values.reshape(-1, 1)

    imp = SimpleImputer(strategy="mean")
    xs = StandardScaler(with_mean=True, with_std=True)
    ys = StandardScaler(with_mean=True, with_std=True)

    Xtr = xs.fit_transform(imp.fit_transform(Xtr_df))
    Xva = xs.transform(imp.transform(Xva_df))

    ytr_s = ys.fit_transform(ytr).ravel()
    yva_s = ys.transform(yva).ravel()

    sd = Xtr.std(axis=0)
    keep = sd > 1e-12
    kept_idx = np.where(keep)[0]
    kept_feats = [feature_names[i] for i in kept_idx]

    Xtr = Xtr[:, keep]
    Xva = Xva[:, keep]
    return Xtr, ytr_s, Xva, yva_s, ys, kept_feats, kept_idx

# -------------------------
# 5) Screening: top-200 abs corr (TRAIN-only)
# -------------------------
def corr_abs_all(Xtr, ytr_s):
    y = np.asarray(ytr_s).ravel()
    Xc = Xtr - Xtr.mean(axis=0, keepdims=True)
    yc = y - y.mean()
    denom = (np.sqrt((Xc * Xc).sum(axis=0)) * np.sqrt((yc * yc).sum()) + 1e-12)
    corr = (Xc.T @ yc) / denom
    return np.abs(corr)

def topk_corr_indices(Xtr, ytr_s, k=200):
    score = corr_abs_all(Xtr, ytr_s)
    k = int(min(k, Xtr.shape[1]))
    idx = np.argsort(score)[::-1][:k]
    return np.sort(idx), score

# -------------------------
# 6) Lasso + LLM-Lasso
# -------------------------
LASSO_ALPHAS = np.logspace(-4, 0.5, 30)
POWERS = [0.0, 0.5, 1.0, 2.0, 3.0]

def fit_lasso_fold(Xtr, ytr, Xva, alphas, inner_cv=5, seed=0):
    model = LassoCV(alphas=alphas, cv=inner_cv, random_state=seed, max_iter=20000, fit_intercept=True)
    model.fit(Xtr, ytr)
    return {"pred": model.predict(Xva), "coef": model.coef_.copy(), "alpha": float(model.alpha_)}

def _cv_mse_at_alpha(model: LassoCV) -> float:
    i_star = np.where(model.alphas_ == model.alpha_)[0]
    if len(i_star) == 0:
        return float(np.mean(np.min(model.mse_path_, axis=0)))
    return float(np.mean(model.mse_path_[i_star[0], :]))

def fit_llm_lasso_fold(Xtr, ytr, Xva, imp_01_1_feats, powers, alphas, inner_cv=5, seed=0):
    best = {"cv_score": np.inf}
    for pwr in powers:
        pen = penalty_factors_from_importance(imp_01_1_feats, power=float(pwr))
        Xtr_w = Xtr / pen
        Xva_w = Xva / pen

        model = LassoCV(alphas=alphas, cv=inner_cv, random_state=seed, max_iter=20000, fit_intercept=True)
        model.fit(Xtr_w, ytr)
        cv_mse = _cv_mse_at_alpha(model)

        beta = model.coef_ / pen
        pred_va = float(model.intercept_) + Xva.dot(beta)

        if cv_mse < best["cv_score"]:
            best = {
                "pred": pred_va,
                "coef": beta.copy(),
                "alpha": float(model.alpha_),
                "power": float(pwr),
                "cv_score": float(cv_mse),
                "pen": pen.copy(),
            }
    return best

# -------------------------
# 7) SSVS Gibbs + K-only inner CV + Top-K refit
# -------------------------
def importance_to_pi_raw(imp_01_1, pi_min=0.001, pi_max=0.90, gamma=6.0):
    imp = np.asarray(imp_01_1, float)
    lo, hi = float(np.min(imp)), float(np.max(imp))
    if hi - lo < 1e-12:
        z = np.full_like(imp, 0.5, dtype=float)
    else:
        z = (imp - lo) / (hi - lo)
    z = np.clip(z, 0.0, 1.0) ** float(gamma)
    return pi_min + (pi_max - pi_min) * z

def rescale_pi_to_mean_logit(pi_raw, target_mean, eps=1e-6):
    pi_raw = np.clip(np.asarray(pi_raw, float), eps, 1 - eps)
    z = np.log(pi_raw / (1 - pi_raw))
    lo, hi = -20.0, 20.0
    for _ in range(60):
        mid = 0.5 * (lo + hi)
        m = float(np.mean(1.0 / (1.0 + np.exp(-(z + mid)))))
        if m > target_mean:
            hi = mid
        else:
            lo = mid
    a = 0.5 * (lo + hi)
    pi = 1.0 / (1.0 + np.exp(-(z + a)))
    return np.clip(pi, eps, 1 - eps)

def make_pi_baseline(p_used, K_expected):
    pi0 = np.full(p_used, float(K_expected) / max(p_used, 1), dtype=float)
    return np.clip(pi0, 1e-6, 1 - 1e-6)

def make_pi_llm(imp_vec, K_expected, gamma_llm, pi_min=0.001, pi_max=0.90):
    pi_raw = importance_to_pi_raw(imp_vec, pi_min=pi_min, pi_max=pi_max, gamma=float(gamma_llm))
    target_mean = float(K_expected) / max(len(imp_vec), 1)
    return rescale_pi_to_mean_logit(pi_raw, target_mean)

def ssvs_gibbs_coordinate_sigma_scaled(
    X, y, pi,
    tau0=0.01, tau1=1.0,
    a0=1e-2, b0=1e-2,
    n_iter=2200, burn=1000, thin=2,
    seed=0
):
    rng = np.random.default_rng(seed)
    n, p = X.shape
    eps = 1e-12
    pi = np.clip(np.asarray(pi, float).ravel(), 1e-6, 1 - 1e-6)

    beta = np.zeros(p, dtype=float)
    sigma2 = 1.0
    gamma = rng.binomial(1, pi, size=p).astype(int)

    r = y.copy()
    x2 = (X * X).sum(axis=0) + eps

    beta_sum = np.zeros(p, dtype=float)
    gamma_sum = np.zeros(p, dtype=float)
    n_save = 0

    for it in range(int(n_iter)):
        v1 = sigma2 * (tau1 ** 2)
        v0 = sigma2 * (tau0 ** 2)

        logp1 = np.log(pi + eps) - 0.5 * np.log(v1 + eps) - (beta ** 2) / (2.0 * (v1 + eps))
        logp0 = np.log(1.0 - pi + eps) - 0.5 * np.log(v0 + eps) - (beta ** 2) / (2.0 * (v0 + eps))
        prob1 = 1.0 / (1.0 + np.exp(logp0 - logp1))
        gamma = rng.binomial(1, prob1).astype(int)

        for j in range(p):
            r += X[:, j] * beta[j]
            tau_j = tau1 if gamma[j] == 1 else tau0
            prior_var = sigma2 * (tau_j ** 2) + eps
            post_var = 1.0 / (x2[j] / sigma2 + 1.0 / prior_var)
            post_mean = post_var * (X[:, j].dot(r) / sigma2)
            beta[j] = rng.normal(post_mean, np.sqrt(post_var))
            r -= X[:, j] * beta[j]

        rss = float((r * r).sum())
        shape = a0 + n / 2.0
        scale = b0 + 0.5 * rss
        sigma2 = 1.0 / rng.gamma(shape, 1.0 / scale)

        if it >= burn and ((it - burn) % thin == 0):
            beta_sum += beta
            gamma_sum += gamma
            n_save += 1

    beta_mean = beta_sum / max(n_save, 1)
    pip = gamma_sum / max(n_save, 1)
    return beta_mean, pip

def fit_ssvs_single(Xtr, ytr, pi, tau0, tau1, n_iter, burn, thin, seed):
    beta, pip = ssvs_gibbs_coordinate_sigma_scaled(
        Xtr, ytr, pi,
        tau0=float(tau0), tau1=float(tau1),
        n_iter=int(n_iter), burn=int(burn), thin=int(thin),
        seed=int(seed)
    )
    return beta, pip

def inner_cv_pick_ssvs_K_only(
    Xtr_s, ytr_s, imp_scr,
    is_llm: bool,
    K_grid,
    tau0=0.01,
    tau1_fixed=1.0,
    gamma_llm_fixed=6.0,
    inner_splits=2,
    n_iter=900, burn=400, thin=2,
    base_seed=0
):
    n = Xtr_s.shape[0]
    kf_in = KFold(n_splits=int(inner_splits), shuffle=True, random_state=int(base_seed))
    best = {"cv_mse": np.inf}

    for Kexp in K_grid:
        mses = []
        for i, (i_tr, i_va) in enumerate(kf_in.split(np.arange(n))):
            Xin_tr, yin_tr = Xtr_s[i_tr], ytr_s[i_tr]
            Xin_va, yin_va = Xtr_s[i_va], ytr_s[i_va]

            if is_llm:
                pi = make_pi_llm(imp_scr, int(Kexp), gamma_llm=float(gamma_llm_fixed))
                g_term = int(10 * float(gamma_llm_fixed))
            else:
                pi = make_pi_baseline(Xtr_s.shape[1], int(Kexp))
                g_term = 0

            seed = (
                1_000_000
                + int(base_seed) * 10_000
                + int(Kexp) * 101
                + int(round(100 * float(tau1_fixed))) * 17
                + 97 * i
                + g_term
            )

            beta, _ = fit_ssvs_single(
                Xin_tr, yin_tr, pi,
                tau0=tau0, tau1=float(tau1_fixed),
                n_iter=n_iter, burn=burn, thin=thin,
                seed=seed
            )
            pred = Xin_va @ beta
            mses.append(mean_squared_error(yin_va, pred))

        cv_mse = float(np.mean(mses))
        if cv_mse < best["cv_mse"]:
            best = {
                "cv_mse": cv_mse,
                "K_expected": int(Kexp),
                "tau1": float(tau1_fixed),
                "gamma_llm": (None if not is_llm else float(gamma_llm_fixed)),
            }
    return best

def topk_pip_refit_beta(Xtr, ytr, pip, k, ridge=1e-6):
    pip = np.asarray(pip, float).ravel()
    p = pip.shape[0]
    k = int(max(0, min(int(k), p)))
    beta_full = np.zeros(p, dtype=float)
    if k == 0:
        return beta_full, 0
    S = np.argsort(pip)[::-1][:k]
    S = np.sort(S)
    Xs = Xtr[:, S]
    XtX = Xs.T @ Xs + float(ridge) * np.eye(len(S))
    Xty = Xs.T @ ytr
    beta_s = np.linalg.solve(XtX, Xty)
    beta_full[S] = beta_s
    return beta_full, int(len(S))

# -------------------------
# 8) Settings
# -------------------------
K_FOLDS = 10
N_REPEATS = 5
BASE_SEED = 0

TOPK = 200

TAU0 = 0.01
TAU1_FIXED = 1.0
GAMMA_LLM_FIXED = 6.0

OUT_NITER = 2200
OUT_BURN  = 1000
OUT_THIN  = 2

IN_NITER = 900
IN_BURN  = 400
IN_THIN  = 2
INNER_SPLITS = 2

K_GRID = [10]

assert len(df) >= K_FOLDS, f"Subset too small for {K_FOLDS}-fold CV: n={len(df)}"

# -------------------------
# 9) Run CV + record individual preds
# -------------------------
fold_rows = []
chosen_rows = []
ind_pred_chunks = []  # list of per-fold DataFrames (individual-level)

for rep in range(N_REPEATS):
    kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=BASE_SEED + rep)

    for fold, (tr_idx, va_idx) in enumerate(kf.split(df), start=1):
        df_tr = df.iloc[tr_idx].copy()
        df_va = df.iloc[va_idx].copy()

        Xtr, ytr_s, Xva, yva_s, y_scaler, kept_feats, kept_idx = preprocess_fold(df_tr, df_va, FEATURES)

        # TRAIN-only screening: top-200 corr
        idx_top, _ = topk_corr_indices(Xtr, ytr_s, k=TOPK)

        Xtr_s = Xtr[:, idx_top]
        Xva_s = Xva[:, idx_top]
        feats_s = [kept_feats[i] for i in idx_top]
        p_used = Xtr_s.shape[1]

        imp_scr = np.array([imp_map_full.get(f, 0.5) for f in feats_s], dtype=float)

        # (1) Lasso
        lasso_fit = fit_lasso_fold(
            Xtr_s, ytr_s, Xva_s,
            alphas=LASSO_ALPHAS, inner_cv=5,
            seed=10_000 + rep * 100 + fold
        )

        # (2) LLM-Lasso
        llm_lasso_fit = fit_llm_lasso_fold(
            Xtr_s, ytr_s, Xva_s, imp_scr,
            powers=POWERS, alphas=LASSO_ALPHAS,
            inner_cv=5, seed=20_000 + rep * 100 + fold
        )

        # Inner CV (K-only): baseline SSVS
        best_base = inner_cv_pick_ssvs_K_only(
            Xtr_s, ytr_s, imp_scr,
            is_llm=False,
            K_grid=K_GRID,
            tau0=TAU0,
            tau1_fixed=TAU1_FIXED,
            gamma_llm_fixed=GAMMA_LLM_FIXED,
            inner_splits=INNER_SPLITS,
            n_iter=IN_NITER, burn=IN_BURN, thin=IN_THIN,
            base_seed=30_000 + rep * 100 + fold
        )

        # Inner CV (K-only): LLM SSVS
        best_llm = inner_cv_pick_ssvs_K_only(
            Xtr_s, ytr_s, imp_scr,
            is_llm=True,
            K_grid=K_GRID,
            tau0=TAU0,
            tau1_fixed=TAU1_FIXED,
            gamma_llm_fixed=GAMMA_LLM_FIXED,
            inner_splits=INNER_SPLITS,
            n_iter=IN_NITER, burn=IN_BURN, thin=IN_THIN,
            base_seed=40_000 + rep * 100 + fold
        )

        # Fit outer TRAIN: baseline SSVS
        pi0 = make_pi_baseline(p_used, best_base["K_expected"])
        beta_ss_mean, pip_ss = fit_ssvs_single(
            Xtr_s, ytr_s, pi0,
            tau0=TAU0, tau1=TAU1_FIXED,
            n_iter=OUT_NITER, burn=OUT_BURN, thin=OUT_THIN,
            seed=50_000 + rep * 100 + fold
        )

        # Fit outer TRAIN: LLM SSVS
        pi_llm = make_pi_llm(imp_scr, best_llm["K_expected"], gamma_llm=GAMMA_LLM_FIXED)
        beta_llm_mean, pip_llm = fit_ssvs_single(
            Xtr_s, ytr_s, pi_llm,
            tau0=TAU0, tau1=TAU1_FIXED,
            n_iter=OUT_NITER, burn=OUT_BURN, thin=OUT_THIN,
            seed=60_000 + rep * 100 + fold
        )

        # TRAIN-only refit using Top-K PIPs (K_expected)  [NO pip>=0.5 for prediction]
        beta_ss_refit,  size_base = topk_pip_refit_beta(Xtr_s, ytr_s, pip_ss,  k=best_base["K_expected"], ridge=1e-6)
        beta_llm_refit, size_llm  = topk_pip_refit_beta(Xtr_s, ytr_s, pip_llm, k=best_llm["K_expected"],  ridge=1e-6)

        # In original units
        y_true = y_scaler.inverse_transform(yva_s.reshape(-1, 1)).ravel()
        def inv(pred_s):
            return y_scaler.inverse_transform(np.asarray(pred_s).reshape(-1, 1)).ravel()

        preds = {
            "lasso": inv(lasso_fit["pred"]),
            "llm_lasso": inv(llm_lasso_fit["pred"]),
            "spike_slab_mcmc_fixed_mpm": inv(Xva_s @ beta_ss_refit),
            "llm_spike_slab_mcmc_fixed_mpm": inv(Xva_s @ beta_llm_refit),
        }

        # ---- individual-level table ----
        ind_df = pd.DataFrame({
            "rep": rep,
            "fold": fold,
            "row_index": df_va.index.values,          # original row index within df after subsetting
            ID_COL: df_va[ID_COL].values,
            "truth": y_true,
            "lasso_pred": preds["lasso"],
            "llm_lasso_pred": preds["llm_lasso"],
            "spike_slab_mcmc_fixed_mpm_pred": preds["spike_slab_mcmc_fixed_mpm"],
            "llm_spike_slab_mcmc_fixed_mpm_pred": preds["llm_spike_slab_mcmc_fixed_mpm"],
        })
        # squared errors
        ind_df["lasso_sq_error"] = (ind_df["lasso_pred"] - ind_df["truth"]) ** 2
        ind_df["llm_lasso_sq_error"] = (ind_df["llm_lasso_pred"] - ind_df["truth"]) ** 2
        ind_df["spike_slab_mcmc_fixed_mpm_sq_error"] = (ind_df["spike_slab_mcmc_fixed_mpm_pred"] - ind_df["truth"]) ** 2
        ind_df["llm_spike_slab_mcmc_fixed_mpm_sq_error"] = (ind_df["llm_spike_slab_mcmc_fixed_mpm_pred"] - ind_df["truth"]) ** 2

        ind_pred_chunks.append(ind_df)

        # ---- fold-level perf summary ----
        for m, yhat in preds.items():
            fold_rows.append({
                "rep": rep,
                "fold": fold,
                "method": m,
                "n_train": int(len(df_tr)),
                "n_val": int(len(df_va)),
                "p_used": int(p_used),
                "mse": float(mean_squared_error(y_true, yhat)),
                "r2": float(r2_score(y_true, yhat)),
            })

        chosen_rows.append({
            "rep": rep, "fold": fold, "p_used": int(p_used),
            "base_K": best_base["K_expected"], "base_cv_mse": best_base["cv_mse"],
            "llm_K": best_llm["K_expected"], "llm_cv_mse": best_llm["cv_mse"],
            "tau1_fixed": float(TAU1_FIXED),
            "gamma_llm_fixed": float(GAMMA_LLM_FIXED),
            "refit_rule": "Top-K PIP refit (K_expected)",
            "refit_size_base": int(size_base),
            "refit_size_llm": int(size_llm),
            "llm_lasso_power": float(llm_lasso_fit["power"]),
            "lasso_alpha": float(lasso_fit["alpha"]),
            "llm_lasso_alpha": float(llm_lasso_fit["alpha"]),
        })

        print(
            f"done rep={rep} fold={fold} | n={len(df_tr)}/{len(df_va)} p_used={p_used} "
            f"| base(K={best_base['K_expected']},refit={size_base}) "
            f"| llm(K={best_llm['K_expected']},refit={size_llm})",
            flush=True
        )

    print(f"[repeat {rep+1}/{N_REPEATS}] done", flush=True)

perf   = pd.DataFrame(fold_rows)
chosen = pd.DataFrame(chosen_rows)
indiv  = pd.concat(ind_pred_chunks, ignore_index=True)

print("\n=== Mean outer-fold performance ===")
print(perf.groupby("method")[["mse", "r2"]].mean().sort_values("mse"))

# -------------------------
# 10) Performance SE (fold-level)
#
# -------------------------
def se(x):
    x = pd.to_numeric(x, errors="coerce").dropna().to_numpy()
    n = len(x)
    if n <= 1:
        return np.nan
    return float(np.std(x, ddof=1) / np.sqrt(n))

def summarize_perf_with_se(perf_df):
    out = []
    for method, sub in perf_df.groupby("method", dropna=False):
        row = {"method": method}
        for m in ["mse", "r2"]:
            vals = pd.to_numeric(sub[m], errors="coerce")
            row[f"{m}_n"] = int(vals.notna().sum())
            row[f"{m}_mean"] = float(vals.mean())
            row[f"{m}_std"]  = float(vals.std(ddof=1))
            row[f"{m}_se"]   = se(vals)
        out.append(row)
    return pd.DataFrame(out).sort_values("method").reset_index(drop=True)

perf_se = summarize_perf_with_se(perf)
print("\n=== PERF: mean/std/SE over all outer folds (rep*fold rows) ===")
print(perf_se)

# -------------------------
# 11) Save outputs to Drive
# -------------------------
OUT_PERF   = os.path.join(OUT_DIR, "aki_t60_subset_pvd_smoker_repeated10fold_perf_4methods_top200corr_fixedTauGamma_refitTopK.csv")
OUT_CHOSEN = os.path.join(OUT_DIR, "aki_t60_subset_pvd_smoker_repeated10fold_chosen_hparams_fixedTauGamma_refitTopK.csv")
OUT_INDIV  = os.path.join(OUT_DIR, "aki_t60_subset_pvd_smoker_individual_level_predictions_and_sq_errors.csv")
OUT_PERFSE = os.path.join(OUT_DIR, "aki_t60_subset_pvd_smoker_perf_summary_with_se.csv")

perf.to_csv(OUT_PERF, index=False)
chosen.to_csv(OUT_CHOSEN, index=False)
indiv.to_csv(OUT_INDIV, index=False)
perf_se.to_csv(OUT_PERFSE, index=False)
